<a href="https://colab.research.google.com/github/valogonor/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/LS_DS_231_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - Logistic Regression

Logistic regression is the baseline for classification models, as well as a handy way to predict probabilities (since those too live in the unit interval). While relatively simple, it is also the foundation for more sophisticated classification techniques such as neural networks (many of which can effectively be thought of as networks of logistic models).

## Lecture - Where Linear goes Wrong
### Return of the Titanic 🚢

You've likely already explored the rich dataset that is the Titanic - let's use regression and try to predict survival with it. The data is [available from Kaggle](https://www.kaggle.com/c/titanic/data), so we'll also play a bit with [the Kaggle API](https://github.com/Kaggle/kaggle-api).

In [1]:
!pip install kaggle

In [2]:
# Note - you'll also have to sign up for Kaggle and authorize the API
# https://github.com/Kaggle/kaggle-api#api-credentials

# This essentially means uploading a kaggle.json file
# For Colab we can have it in Google Drive
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

# You also have to join the Titanic competition to have access to the data
# !kaggle competitions download -c titanic

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [3]:
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 18.9MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 23.3MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.12MB/s]


In [4]:
# How would we try to do this with linear regression?
import pandas as pd

train_df = pd.read_csv('train.csv').dropna()
test_df = pd.read_csv('test.csv').dropna()  # Unlabeled, for Kaggle submission

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


In [5]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000,183.000000
mean,455.366120,0.672131,1.191257,35.674426,0.464481,0.475410,78.682469
std,247.052476,0.470725,0.515187,15.643866,0.644159,0.754617,76.347843
min,2.000000,0.000000,1.000000,0.920000,0.000000,0.000000,0.000000
25%,263.500000,0.000000,1.000000,24.000000,0.000000,0.000000,29.700000
50%,457.000000,1.000000,1.000000,36.000000,0.000000,0.000000,57.000000
75%,676.000000,1.000000,1.000000,47.500000,1.000000,1.000000,90.000000
max,890.000000,1.000000,3.000000,80.000000,3.000000,4.000000,512.329200


In [6]:
from sklearn.linear_model import LinearRegression

X = train_df[['Pclass', 'Age', 'Fare']]
y = train_df.Survived

linear_reg = LinearRegression().fit(X, y)
linear_reg.score(X, y)

0.08389810726550917

In [7]:
linear_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([0.79543117, 0.58610823, 0.67595121, 0.793829  , 0.62090522,
       0.72542107, 0.59848968, 0.58734245, 0.48567063, 0.77627736,
       0.84211887, 0.57052439, 0.7754689 , 0.96621114, 0.70287941,
       0.57673837, 0.72321391, 0.75894755, 0.77968041, 0.50246003,
       0.49858077, 0.7474959 , 0.3542282 , 0.61648435, 0.71300224,
       0.66294608, 0.53175333, 0.77397395, 0.68419387, 0.68395536,
       0.52041202, 0.56814038, 0.79586606, 0.81372012, 0.61068545,
       0.57260627, 0.52525981, 0.58055388, 0.45584728, 0.67976208,
       0.8226707 , 0.84286197, 0.96189157, 0.66724612, 0.68589478,
       0.61846513, 0.63455044, 0.68275686, 0.65738372, 0.45198998,
       0.59988596, 0.63845908, 0.63132487, 0.7888473 , 0.60126246,
       0.79714045, 0.78713803, 0.54643775, 0.42823635, 0.7711724 ,
       0.53552976, 0.55608044, 0.54480459, 0.57031915, 0.65080369,
       0.77958926, 0.6371013 , 0.70993488, 0.71493598, 0.60375943,
       0.54407206, 0.48186138, 0.76576089, 0.75456305, 0.53968

In [8]:
linear_reg.coef_

array([-0.08596295, -0.00829314,  0.00048775])

In [9]:
import numpy as np

test_case = np.array([[1, 5, 500]])  # Rich 5-year old in first class
linear_reg.predict(test_case)

array([1.14845883])

In [10]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7103825136612022

In [11]:
log_reg.predict(test_df[['Pclass', 'Age', 'Fare']])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [12]:
log_reg.predict(test_case)[0]

1

In [13]:
help(log_reg.predict)

Help on method predict in module sklearn.linear_model.base:

predict(X) method of sklearn.linear_model.logistic.LogisticRegression instance
    Predict class labels for samples in X.
    
    Parameters
    ----------
    X : array_like or sparse matrix, shape (n_samples, n_features)
        Samples.
    
    Returns
    -------
    C : array, shape [n_samples]
        Predicted class label per sample.



In [14]:
log_reg.predict_proba(test_case)[0]

array([0.02485552, 0.97514448])

In [15]:
# What's the math?
log_reg.coef_

array([[-0.0455017 , -0.02912513,  0.0048037 ]])

In [16]:
log_reg.intercept_

array([1.45878264])

In [0]:
# The logistic sigmoid "squishing" function, implemented to accept numpy arrays
def sigmoid(x):
  return 1 / (1 + np.e**(-x))

In [18]:
sigmoid(log_reg.intercept_ + np.dot(log_reg.coef_, np.transpose(test_case)))

array([[0.97514448]])

So, clearly a more appropriate model in this situation! For more on the math, [see this Wikipedia example](https://en.wikipedia.org/wiki/Logistic_regression#Probability_of_passing_an_exam_versus_hours_of_study).

For live - let's tackle [another classification dataset on absenteeism](http://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work) - it has 21 classes, but remember, scikit-learn LogisticRegression automatically handles more than two classes. How? By essentially treating each label as different (1) from some base class (0).

In [0]:
# Live - let's try absenteeism!

## Assignment - real-world classification

We're going to check out a larger dataset - the [FMA Free Music Archive data](https://github.com/mdeff/fma). It has a selection of CSVs with metadata and calculated audio features that you can load and try to use to classify genre of tracks. To get you started:

In [2]:
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
!unzip fma_metadata.zip

--2019-01-23 05:01:57--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.13, 2001:620:5ca1:2ff::ce53
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip’

fma_metadata.zip    100%[===================>] 341.81M  17.0MB/s    in 37s     

2019-01-23 05:02:35 (9.17 MB/s) - ‘fma_metadata.zip’ saved [358412441/358412441]

Archive:  fma_metadata.zip
 bunzipping: fma_metadata/README.txt  
 bunzipping: fma_metadata/checksums  
 bunzipping: fma_metadata/not_found.pickle  
 bunzipping: fma_metadata/raw_genres.csv  
 bunzipping: fma_metadata/raw_albums.csv  
 bunzipping: fma_metadata/raw_artists.csv  
 bunzipping: fma_metadata/raw_tracks.csv  
 bunzipping: fma_metadata/tracks.csv  
 bunzipping: fma_metadata/genres.csv  
 bunzipping: fma_metadata/raw_echonest.cs

In [71]:
import pandas as pd

# Take columns names from first row
tracks = pd.read_csv('fma_metadata/tracks.csv', header=1)

# Manually name the first column, and remove the first line where that name used to be.
tracks.rename(columns={'Unnamed: 0':'track_id'}, inplace=True)
tracks.drop(index=0, inplace=True)
tracks.head()

# Put the genre_top column at the beginning
tracks = tracks.reindex(columns=(['genre_top'] + list([a for a in tracks.columns if a != 'genre_top'])))

# Remove any columns that don't have an entry for genre_top
tracks = tracks.dropna(subset=['genre_top'])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [72]:
print(tracks.shape)
tracks.head()

(49598, 53)


,genre_top,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,tags,title,tracks,type,active_year_begin,active_year_end,associated_labels,bio,comments.1,date_created.1,favorites.1,id.1,latitude,location,longitude,members,name,related_projects,tags.1,website,wikipedia_page,split,subset,bit_rate,comments.2,composer,date_created.2,date_recorded,duration,favorites.2,genres,genres_all,information.1,interest,language_code,license,listens.1,lyricist,number,publisher,tags.2,title.1
1,Hip-Hop,2,0.0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4.0,1.0,<p></p>,6073.0,NaN,[],AWOL - A Way Of Life,7.0,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0.0,2008-11-26 01:42:32,9.0,1.0,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000.0,0.0,NaN,2008-11-26 01:48:12,2008-11-26 00:00:00,168.0,2.0,[21],[21],NaN,4656.0,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1293.0,NaN,3.0,NaN,[],Food
2,Hip-Hop,3,0.0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4.0,1.0,<p></p>,6073.0,NaN,[],AWOL - A Way Of Life,7.0,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0.0,2008-11-26 01:42:32,9.0,1.0,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000.0,0.0,NaN,2008-11-26 01:48:14,2008-11-26 00:00:00,237.0,1.0,[21],[21],NaN,1470.0,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,514.0,NaN,4.0,NaN,[],Electric Ave
3,Hip-Hop,5,0.0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4.0,1.0,<p></p>,6073.0,NaN,[],AWOL - A Way Of Life,7.0,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0.0,2008-11-26 01:42:32,9.0,1.0,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,small,256000.0,0.0,NaN,2008-11-26 01:48:20,2008-11-26 00:00:00,206.0,6.0,[21],[21],NaN,1933.0,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,1151.0,NaN,6.0,NaN,[],This World
4,Pop,10,0.0,2008-11-26 01:45:08,2008-02-06 00:00:00,NaN,4.0,6.0,NaN,47632.0,NaN,[],Constant Hitmaker,2.0,Album,NaN,NaN,"Mexican Summer, Richie Records, Woodsist, Skul...","<p><span style=""font-family:Verdana, Geneva, A...",3.0,2008-11-26 01:42:55,74.0,6.0,NaN,NaN,NaN,"Kurt Vile, the Violators",Kurt Vile,NaN,"['philly', 'kurt vile']",http://kurtvile.com,NaN,training,small,192000.0,0.0,Kurt Vile,2008-11-25 17:49:06,2008-11-26 00:00:00,161.0,178.0,[10],[10],NaN,54881.0,en,Attribution-NonCommercial-NoDerivatives (aka M...,50135.0,NaN,1.0,NaN,[],Freeway
10,Hip-Hop,134,0.0,2008-11-26 01:44:45,2009-01-05 00:00:00,NaN,4.0,1.0,<p></p>,6073.0,NaN,[],AWOL - A Way Of Life,7.0,Album,2006-01-01 00:00:00,NaN,NaN,"<p>A Way Of Life, A Collective of Hip-Hop from...",0.0,2008-11-26 01:42:32,9.0,1.0,40.058324,New Jersey,-74.405661,"Sajje Morocco,Brownbum,ZawidaGod,Custodian of ...",AWOL,The list of past projects is 2 long but every1...,['awol'],http://www.AzillionRecords.blogspot.com,NaN,training,medium,256000.0,0.0,NaN,2008-11-26 01:43:19,2008-11-26 00:00:00,207.0,3.0,[21],[21],NaN,1126.0,en,Attribution-NonCommercial-ShareAlike 3.0 Inter...,943.0,NaN,5.0,NaN,[],Street Music


In [73]:
garbage_columns = ['track_id','id', 'information','comments.1','title','bio',
                   'members','website','wikipedia_page','split','subset',
                   'comments.2','genres','genres_all','information.1','license','title.1']

tracks2 = tracks.drop(columns=garbage_columns)
tracks2.shape

(49598, 36)

In [74]:
# Percentace of rows with null values for each column
tracks2.isnull().sum() / len(tracks2) * 100

genre_top             0.000000
comments              0.000000
date_created          2.119037
date_released        32.432759
engineer             82.033550
favorites             0.000000
listens               0.000000
producer             79.382233
tags                  0.000000
tracks                0.000000
type                  4.127183
active_year_begin    80.680673
active_year_end      95.165128
associated_labels    87.614420
date_created.1        0.433485
favorites.1           0.000000
id.1                  0.000000
latitude             57.699907
location             35.259486
longitude            57.699907
name                  0.000000
related_projects     89.183032
tags.1                0.000000
bit_rate              0.000000
composer             97.272067
date_created.2        0.000000
date_recorded        91.429090
duration              0.000000
favorites.2           0.000000
interest              0.000000
language_code        81.081898
listens.1             0.000000
lyricist

In [75]:
tracks_numeric = tracks2.select_dtypes('number')
tracks_numeric.isnull().sum()

comments           0
favorites          0
listens            0
tracks             0
favorites.1        0
id.1               0
latitude       28618
longitude      28618
bit_rate           0
duration           0
favorites.2        0
interest           0
listens.1          0
number             0
dtype: int64

In [76]:
tracks_numeric = tracks_numeric.dropna(axis='columns')
tracks_numeric.describe()

,comments,favorites,listens,tracks,favorites.1,id.1,bit_rate,duration,favorites.2,interest,listens.1,number
count,49598.000000,49598.000000,4.959800e+04,49598.000000,49598.000000,49598.000000,49598.000000,49598.000000,49598.000000,4.959800e+04,49598.000000,49598.000000
mean,0.333179,1.145449,1.997469e+04,21.763035,16.721602,11400.403786,260278.657930,268.627263,2.381447,2.523905e+03,1586.328380,8.545607
std,1.312889,2.463242,5.736980e+04,51.492489,58.966302,7046.466624,65663.862632,284.327919,11.147578,1.980285e+04,6039.952955,16.988260
min,-1.000000,-1.000000,-1.000000e+00,-1.000000,-1.000000,1.000000,-1.000000,0.000000,0.000000,2.000000e+00,1.000000,0.000000
25%,0.000000,0.000000,2.602000e+03,7.000000,1.000000,5449.000000,192000.000000,146.000000,0.000000,4.560000e+02,212.000000,2.000000
50%,0.000000,0.000000,6.092000e+03,11.000000,4.000000,11384.000000,256000.000000,211.000000,1.000000,9.380000e+02,520.000000,5.000000
75%,0.000000,1.000000,1.621900e+04,17.000000,12.000000,17450.000000,320000.000000,299.000000,2.000000,2.091000e+03,1321.000000,9.000000
max,17.000000,40.000000,1.193803e+06,652.000000,963.000000,24357.000000,448000.000000,11030.000000,1482.000000,3.293557e+06,543252.000000,255.000000


In [79]:
from sklearn.linear_model import LogisticRegression

y = tracks2['genre_top']
X = tracks_numeric

# Split into train and test groups
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
                                                   random_state=42)
log_reg = LogisticRegression(multi_class='ovr',
                            solver='liblinear',
                            max_iter=500).fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.3859429815718376

In [80]:
# Creating DF of abs(coeffs)
model_coefs = pd.DataFrame(log_reg.coef_, 
                           columns = tracks_numeric.columns,
                           index = log_reg.classes_).abs()

model_coefs

,comments,favorites,listens,tracks,favorites.1,id.1,bit_rate,duration,favorites.2,interest,listens.1,number
Blues,0.002216,0.002138,2.511547e-06,0.047509,0.001482,1.187876e-04,1.721635e-05,0.002742,0.027618,0.000013,0.000007,0.041422
Classical,0.000159,0.001114,8.790122e-06,0.006650,0.001246,2.403053e-05,1.435025e-05,0.000333,0.000684,0.000004,0.000018,0.000194
Country,0.000351,0.002537,4.004578e-06,0.101806,0.001185,3.295843e-05,1.071885e-05,0.007310,0.004041,0.000025,0.000021,0.025974
Easy Listening,0.001077,0.001079,1.777963e-05,0.122904,0.008803,9.941423e-05,1.360111e-05,0.006531,0.006450,0.001620,0.001229,0.058545
Electronic,0.000015,0.000032,1.193056e-06,0.001375,0.000003,1.645735e-05,3.650037e-06,0.000734,0.000093,0.000004,0.000032,0.000516
Experimental,0.000361,0.000098,2.541137e-05,0.018514,0.003923,1.140080e-05,4.382287e-06,0.001105,0.001739,0.000066,0.000383,0.008168
Folk,0.000004,0.000265,1.012971e-05,0.003541,0.005108,1.463720e-05,7.928983e-06,0.001452,0.000336,0.000130,0.000050,0.003063
Hip-Hop,0.000029,0.000075,1.180461e-07,0.005405,0.000685,2.186679e-05,3.984224e-06,0.004811,0.000176,0.000079,0.000082,0.000897
Instrumental,0.001284,0.000443,7.039339e-06,0.017911,0.007066,4.661808e-05,1.205491e-05,0.002105,0.001168,0.000617,0.000735,0.001257
International,0.001948,0.001924,8.294091e-06,0.059855,0.018524,2.059851e-05,9.259026e-06,0.000504,0.003891,0.000005,0.000002,0.022770


In [81]:
model_coefs.mean().sort_values(ascending=False)

tracks         0.035001
number         0.020965
favorites.1    0.012119
favorites.2    0.006050
duration       0.002640
favorites      0.001162
comments       0.000979
listens.1      0.000222
interest       0.000205
id.1           0.000043
bit_rate       0.000010
listens        0.000010
dtype: float64

In [82]:
tracks_categorical = tracks2.select_dtypes(exclude='number')
tracks_categorical.columns

Index(['genre_top', 'date_created', 'date_released', 'engineer', 'producer',
       'tags', 'type', 'active_year_begin', 'active_year_end',
       'associated_labels', 'date_created.1', 'location', 'name',
       'related_projects', 'tags.1', 'composer', 'date_created.2',
       'date_recorded', 'language_code', 'lyricist', 'publisher', 'tags.2'],
      dtype='object')

In [85]:
date_columns = ['date_created', 'date_released','active_year_begin', 
               'active_year_end','date_created.1','date_created.2',
               'date_recorded']
tracks_categorical = tracks_categorical.drop(columns=date_columns)
tracks_categorical.describe()

,genre_top,engineer,producer,tags,type,associated_labels,location,name,related_projects,tags.1,composer,language_code,lyricist,publisher,tags.2
count,49598,8911,10226,49598,47551,6143,32110,49598,5365,49598,1353,9383,100,410,49598
unique,16,387,460,993,4,528,1564,9070,516,8833,285,19,27,89,1003
top,Rock,Ernie Indradat,Joe Belock,[],Album,"Care in the Community Recordings, Gagarin Reco...","Brooklyn, NY",James Kibbie,"Drive Like Jehu, Hot Snakes, Pitchfork, Edsel,...",[],J.S. Bach,en,Wesley Willis,www.headphonica.com,[]
freq,14182,602,700,40974,39140,168,1499,319,126,1233,346,9048,13,55,40854


This is the biggest data you've played with so far, and while it does generally fit in Colab, it can take awhile to run. That's part of the challenge!

Your tasks:
- Clean up the variable names in the dataframe
- Use logistic regression to fit a model predicting (primary/top) genre
- Inspect, iterate, and improve your model
- Answer the following questions (written, ~paragraph each):
  - What are the best predictors of genre?
  - What information isn't very useful for predicting genre?
  - What surprised you the most about your results?

*Important caveats*:
- This is going to be difficult data to work with - don't let the perfect be the enemy of the good!
- Be creative in cleaning it up - if the best way you know how to do it is download it locally and edit as a spreadsheet, that's OK!
- If the data size becomes problematic, consider sampling/subsetting
- You do not need perfect or complete results - just something plausible that runs, and that supports the reasoning in your written answers

If you find that fitting a model to classify *all* genres isn't very good, it's totally OK to limit to the most frequent genres, or perhaps trying to combine or cluster genres as a preprocessing step. Even then, there will be limits to how good a model can be with just this metadata - if you really want to train an effective genre classifier, you'll have to involve the other data (see stretch goals).

This is real data - there is no "one correct answer", so you can take this in a variety of directions. Just make sure to support your findings, and feel free to share them as well! This is meant to be practice for dealing with other "messy" data, a common task in data science.

**What are the best predictors of genre?**

I found that the best predictors of genre are tracks, number, and favorites.1.

**What information isn't very useful for predicting genre?**

listens, bit_rate, id.1

**What surprised you the most about your results?**

I'm surprised pop music is not the most popular genre.


## Resources and stretch goals

- Check out the other .csv files from the FMA dataset, and see if you can join them or otherwise fit interesting models with them
- [Logistic regression from scratch in numpy](https://blog.goodaudience.com/logistic-regression-from-scratch-in-numpy-5841c09e425f) - if you want to dig in a bit more to both the code and math (also takes a gradient descent approach, introducing the logistic loss function)
- Create a visualization to show predictions of your model - ideally show a confidence interval based on error!
- Check out and compare classification models from scikit-learn, such as [SVM](https://scikit-learn.org/stable/modules/svm.html#classification), [decision trees](https://scikit-learn.org/stable/modules/tree.html#classification), and [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html). The underlying math will vary significantly, but the API (how you write the code) and interpretation will actually be fairly similar.
- Sign up for [Kaggle](https://kaggle.com), and find a competition to try logistic regression with
- (Not logistic regression related) If you enjoyed the assignment, you may want to read up on [music informatics](https://en.wikipedia.org/wiki/Music_informatics), which is how those audio features were actually calculated. The FMA includes the actual raw audio, so (while this is more of a longterm project than a stretch goal, and won't fit in Colab) if you'd like you can check those out and see what sort of deeper analysis you can do.